# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../output_data/cities.csv"
cities_df = pd.read_csv(csv_path)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Harper,4.3750,-7.7169,78.75,87,29,7.52,LR,1644455923
1,Jamestown,42.0970,-79.2353,42.39,37,0,11.50,US,1644455914
2,Vuktyl,63.8567,57.3094,16.74,88,100,16.31,RU,1644456474
3,Port Lincoln,-34.7333,135.8667,69.13,52,46,9.22,AU,1644456475
4,Lebu,-37.6167,-73.6500,59.85,81,28,21.14,CL,1644455768
...,...,...,...,...,...,...,...,...,...
546,Evensk,61.9500,159.2333,-5.49,74,62,2.91,RU,1644455835
547,Nueve de Julio,-35.4444,-60.8831,69.71,52,0,8.75,AR,1644456616
548,Mogadishu,2.0371,45.3438,76.35,77,100,20.74,SO,1644456059
549,Cuamba,-14.8031,36.5372,68.97,99,100,1.88,MZ,1644456617


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
#Configure gmaps
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(str).tolist()

fig = gmaps.figure()

#Storing max humidity from dataset
max_humid = cities_df['Humidity'].max()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humid, point_radius=5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [63]:
perf_weather = cities_df[(cities_df['Max Temp']>70) & (cities_df['Max Temp']<80) & (cities_df['Wind Speed']<10) & (cities_df['Cloudiness']==0)]
perf_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,lat
126,Formosa,-26.1775,-58.1781,78.78,47,0,5.75,AR,1644456498,NaN
142,Lompoc,34.6391,-120.4579,77.14,26,0,4.61,US,1644455986,NaN
238,Ixtapa,20.7000,-105.2000,72.63,68,0,9.22,MX,1644456329,NaN
263,Agüimes,27.9054,-15.4461,70.66,40,0,1.14,ES,1644456531,NaN
288,Inhambane,-23.8650,35.3833,74.80,83,0,3.42,MZ,1644456537,NaN
377,Lázaro Cárdenas,17.9583,-102.2000,75.96,64,0,7.07,MX,1644456331,NaN
388,Te Anau,-45.4167,167.7167,74.25,41,0,7.23,NZ,1644456564,NaN
423,Bonthe,7.5264,-12.5050,76.82,90,0,2.84,SL,1644456574,NaN
464,Pacific Grove,36.6177,-121.9166,73.35,52,0,0.00,US,1644456586,NaN
478,General Roca,-39.0333,-67.5833,79.20,13,0,7.16,AR,1644456590,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
hotel_df = perf_weather
hotel_name = []

In [65]:
#Storing Latitude and Longitude    
lat = hotel_df['Lat']
lng = hotel_df['Lng']
#print(lat, lng)

In [66]:
#Searching for Hotels within 5000 meters of 
for x in range(len(hotel_df)):

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat.iloc[x]}, {lng.iloc[x]}",
        "type": "hotel",
        "radius": "5000",
        "key": g_key
    }
    #hotel_data = requests.get(base_url, params=params).json()
    # Print the name and address of the first hotel that appears
   
    places_data = requests.get(base_url, params=params).json()
    # Print the name and address of the first bike store that appears
    try:
        hotel_name.append(places_data["results"][0]["name"])
    except:
        print(f" Hotel Search failed")

#print(hotel_name)
#len(hotel_name)

In [67]:
hotel_df['Hotel Name'] = hotel_name

/Users/emmadublin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [69]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))